In [1]:
from fyp_work.paper_to_equation.Gemini.GeminiPredict import GeminiPredict
from sympy import *

c:\Users\kyanj\anaconda3\envs\fyp_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
api_key = "GEMINI_API_KEY"
filename = "Data/HTC_gemini4"
model = "tunedModels/mmltopython4-f3fuppiemnq9"

gp = GeminiPredict(api_key, filename, model)
gp.predict_from_file(filename="Data/HTC_mml") 
gp.save_predictions()

API key loaded successfully!


Generating Predictions:   0%|          | 0/13 [00:00<?, ?row/s]

Generating Predictions: 100%|██████████| 13/13 [00:19<00:00,  1.53s/row]


In [3]:
class SystemOfEquations():
    def __init__(self, filename):
        self.equations = []
        self.symbol_lines = set()
        self.symbols = set()

    def extract_equations(self):
        with open (f"{filename}_predictions.txt", "r") as file:
            for line in file:
                self.equations.append(line)

    def get_equations(self):
        return self.equations

    def parse_equation(self, equation):
        for line in equation.split("\\n"):
            if "Symbol" in line:
                line = line.replace('"', "")
                self.symbol_lines.add(line)
                self.symbols.add(line.split(" = ")[0])
        





def main():
    soe = SystemOfEquations("Data/HTC_gemini4")
    soe.extract_equations()
    # eq = soe.equations[0]
    # soe.parse_equation(eq)
    for eq in soe.equations:
        print(eq)
        soe.parse_equation(eq)
    for symbol in soe.symbols:
        print(symbol)


main()

"h = Symbol('h')\nh_g = Symbol('h_g')\nh_c = Symbol('h_c')\ne = Eq(h, h_g + h_c)"

"h = Symbol('h')\nk = Symbol('k')\nθ = Symbol('θ')\nσ = Symbol('σ')\np = Symbol('p')\nH = Symbol('H')\ne = Eq(h, 1.45*k*tan(θ)/σ*(p/H)**0.985)"

"h = Symbol('h')\nλ_ = Symbol('λ_')\np = Symbol('p')\nC = Symbol('C')\nσ_U = Symbol('σ_U')\nK = Symbol('K')\ne = Eq(h, 8000*λ_*(p/(C*σ_U)*K)**0.86)"

"h = Symbol('h')\nA = Symbol('A')\nB = Symbol('B')\nP = Symbol('P')\ne = Eq(h, A*(1 - exp(-B*P)))"

"h = Symbol('h')\nA = Symbol('A')\nh_f = Symbol('h_f')\nk_f = Symbol('k_f')\nk_t = Symbol('k_t')\nk_w = Symbol('k_w')\ne = Eq(h, (1 - A)/h_f * 2*k_f*k_t*k_w/(2*k_t*k_w - k_w*k_f - k_f*k_t))"

"h = Symbol('h')\nh_a = Symbol('h_a')\nh_c = Symbol('h_c')\nh_l = Symbol('h_l')\ne = Eq(h, h_a + h_c + h_l)"

"h = Symbol('h')\nh_gat = Symbol('h_gat')\nh_c = Symbol('h_c')\ne = Eq(h, h_gat + h_c)\nα = Symbol('α')\nK_st = Symbol('K_st')\nN_P = Symbol('N_P')\nR = Symbol('R')\ne = Eq(h_c, α*K_st*N_P/R)"

"K_st = Symbol('K_st')\nk_